In [1]:
import os
import json

from dotenv import load_dotenv
from docling.document_converter import DocumentConverter

/Users/theo.moreau/Documents/betagouv/aides-agri/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
API_KEY = os.getenv("ALBERT_API_KEY")
ENDPOINT = "https://albert.api.etalab.gouv.fr/v1/agents/completions"
with open("aide_agri.json") as f:
  JSON_SCHEMA = json.load(f)

In [3]:
document_path = [
  "https://aides-redevances.eau-loire-bretagne.fr/files/live/sites/aides-redevances/files/Aides-12prog/Fiches-actions/AGRI_1.pdf",
  "./2017-561_final.pdf"
]

In [4]:
def get_document_context(file_path):
  converter = DocumentConverter()
  doc = converter.convert(file_path).document
  return doc.export_to_text()

In [5]:
import requests

def get_header():
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {API_KEY}",  # Remplace par ta vraie clé
        "Content-Type": "application/json",
    }
    return headers

def get_body_text(model_name, user_message, json_schema, **kwargs):
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "response_format": {
            "type": "text"
        },
        **kwargs
    }

    return payload

def get_body(model_name, user_message, json_schema, **kwargs):
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "response_format": {
            "type": "json_schema",
            "json_schema": json_schema
        },
        **kwargs
    }

    return payload

def albert_structured_output(model_name, user_message, json_schema, **kwargs):
    url = ENDPOINT
    headers = get_header()
    body = get_body(model_name, user_message, json_schema, **kwargs)

    response = requests.post(url, headers=headers, json=body)

    # Vérification du statut et affichage
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur {response.status_code}: {response.text}")

In [6]:
INSTRUCTION_PROMPT = """
Tu es un modèle de langage spécialisé dans l’analyse et la synthèse d’informations.
Ta mission est de produire un résultat strictement structuré en JSON, conforme au schéma spécifié ci-dessous.
- Tu dois suivre fidèlement la structure : aucune information hors du format demandé.
- Ne mets aucun commentaire, texte explicatif ou balise en dehors du JSON.
- Les valeurs doivent respecter le type indiqué (string, integer, boolean, etc.).
- Si une donnée est manquante, utilise null plutôt qu’un texte libre.
"""

In [7]:
responses = []

for idx, path in enumerate(document_path):
  print(f"Analyzing document {idx}")
  document_context = get_document_context(path)
  final_prompt = INSTRUCTION_PROMPT + document_context
  print(len(document_context))
  full_response = albert_structured_output("albert-small", final_prompt, JSON_SCHEMA, temperature=1, max_completion_tokens=300)
  responses.append(full_response)

Analyzing document 0


2025-09-17 16:49:26,183 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-17 16:49:26,277 - INFO - Going to convert document batch...
2025-09-17 16:49:26,278 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-17 16:49:26,293 - INFO - Loading plugin 'docling_defaults'
2025-09-17 16:49:26,294 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-17 16:49:26,306 - INFO - Loading plugin 'docling_defaults'
2025-09-17 16:49:26,310 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-17 16:49:26,478 - INFO - Accelerator device: 'mps'
2025-09-17 16:49:28,034 - INFO - Accelerator device: 'mps'
2025-09-17 16:49:28,975 - INFO - Accelerator device: 'mps'
2025-09-17 16:49:29,319 - INFO - Processing document AGRI_1.pdf
2025-09-17 16:49:32,841 - INFO - Finished converting document AGRI_1.pdf in 7.18 sec.
2025-09-17 16:49:32,859 - WARNING - Parameter `strict_text` has b

9095


2025-09-17 16:49:39,248 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-17 16:49:39,252 - INFO - Going to convert document batch...
2025-09-17 16:49:39,253 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-17 16:49:39,253 - INFO - Accelerator device: 'mps'


Analyzing document 1


2025-09-17 16:49:40,826 - INFO - Accelerator device: 'mps'
2025-09-17 16:49:41,634 - INFO - Accelerator device: 'mps'
2025-09-17 16:49:41,920 - INFO - Processing document 2017-561_final.pdf
2025-09-17 16:49:46,659 - INFO - Finished converting document 2017-561_final.pdf in 7.41 sec.
2025-09-17 16:49:46,669 - WARNING - Parameter `strict_text` has been deprecated and will be ignored.


32449


In [8]:
print(responses[0]["choices"][0]["message"]["content"])

{"titre": "search_datasets", "description": "AGR_1 - Soutenir l'accompagnement collectif et individuel des agriculteurs", "eligibilite": ["public", "privé"], "types_aides": ["prioritaire", "subvention"], "programmes_parents": "AGR_1", "date_cloture": "3006-10-20T18:00:00Z", "date_mise_a_jour": "3006-10-20T18:00:00Z", "id": "S Ranking , GazetteerGeoNames + GazetteerCity Deliminator @DataCard '''elhelperU/", "metadata": {"creation_date": "2020-06-16T13:08:16Z", "publisher": "Agence de l'eau Rhône-Méditerranée", "download_url": "https://data.apollo-gallery.com/datasets?query=label%3AAGR_1", "date_releve": "2020-06-16", "id_dataset": "pbafframtpdb04foundogirosjpgball"} }


In [9]:
print(responses[1]["choices"][0]["message"]["content"])

{"titre": "search_datasets", "description": "Recherche des informations concernant le dispositif d'aide à la réinsertion professionnelle et les formulaires associés en utilisant la fonctionnalité Recherche de jeux de données de la plateforme data.gouv.fr", "eligibilite": 
        [
            "Lorsque l'exploitation a été jugée inapte au redressement sur décision du préfet après avis de la section « agriculteurs en difficulté » de la CDOA ou dans le cadre d'une procédure collective de liquidation judiciaire par le TGI",
            "Le bénéficiaire dispose de 2 ans à compter de la date d'octroi pour cesser définitivement son activité et obtenir le paiement des aides",
            "Peuvent bénéficier de l'ARP, les agriculteurs contraints de cesser leur activité agricole et justifiant de 5 années d'activité agricole au sens de l'art. L. 311-1 du code rural et de la pêche maritime, précédant immédiatement le dépôt de la demande d'ARP, en qualité de :"
        ], "types_aides": 
        [

In [10]:
for idx, res in enumerate(responses):
  print(f"Token number for response {idx}: {res["usage"]["prompt_tokens"]}")

Token number for response 0: 2057
Token number for response 1: 7789


In [11]:
full_response

{'id': 'chatcmpl-20a04590a878429fb90391a2a7d5bd84',
 'object': 'chat.completion',
 'created': 1758120587,
 'model': 'meta-llama/Llama-3.1-8B-Instruct',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{"titre": "search_datasets", "description": "Recherche des informations concernant le dispositif d\'aide à la réinsertion professionnelle et les formulaires associés en utilisant la fonctionnalité Recherche de jeux de données de la plateforme data.gouv.fr", "eligibilite": \n        [\n            "Lorsque l\'exploitation a été jugée inapte au redressement sur décision du préfet après avis de la section « agriculteurs en difficulté » de la CDOA ou dans le cadre d\'une procédure collective de liquidation judiciaire par le TGI",\n            "Le bénéficiaire dispose de 2 ans à compter de la date d\'octroi pour cesser définitivement son activité et obtenir le paiement des aides",\n            "Peuvent bénéficier de l\'ARP, les agriculteurs contraints de cesser leu

In [12]:
structured_output = full_response["choices"][0]["message"]["content"]

In [13]:
print(structured_output)

{"titre": "search_datasets", "description": "Recherche des informations concernant le dispositif d'aide à la réinsertion professionnelle et les formulaires associés en utilisant la fonctionnalité Recherche de jeux de données de la plateforme data.gouv.fr", "eligibilite": 
        [
            "Lorsque l'exploitation a été jugée inapte au redressement sur décision du préfet après avis de la section « agriculteurs en difficulté » de la CDOA ou dans le cadre d'une procédure collective de liquidation judiciaire par le TGI",
            "Le bénéficiaire dispose de 2 ans à compter de la date d'octroi pour cesser définitivement son activité et obtenir le paiement des aides",
            "Peuvent bénéficier de l'ARP, les agriculteurs contraints de cesser leur activité agricole et justifiant de 5 années d'activité agricole au sens de l'art. L. 311-1 du code rural et de la pêche maritime, précédant immédiatement le dépôt de la demande d'ARP, en qualité de :"
        ], "types_aides": 
        [